In [1]:
import xarray as xr
import requests
import netCDF4
import boto3
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
bucket_name = 'noaa-goes16'
product_name = 'ABI-L1b-RadF'
year = 2019
day_of_year = 79
hour = 12
band = 3

In [3]:
def get_s3_keys(bucket, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    s3 = boto3.client('s3')   #
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3.list_objects_v2(**kwargs)    #list all keys
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                yield key
                print(key)

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
            
        print(resp)

ContinuationToken field, which can be passed to the ListObjects API to get the next page of results. By looking for this token, and using it to make another request, we can steadily fetch every key in the bucket.
            
Once we’ve got a response, we extract the continuation token and add it to kwargs, which means we’ll pass it on the next ListObjects call. When we’re on the final page of results, the ContinuationToken field is omitted (because there’s nowhere to continue to) – at which point the KeyError tells us that we should stop looking, and return the list to the user.

In [9]:
keys = get_s3_keys(bucket_name, prefix = product_name+'/'+ str(year) + '/' + str(day_of_year).zfill(3) + '/' + str(hour).zfill(2) + '/OR_'+ 
                                         product_name + '-M3C' + str(band).zfill(2))

key = [key for key in keys][0] # selecting the first measurement taken within the hour

NoCredentialsError: Unable to locate credentials

In [7]:
resp = requests.get('https://' + bucket_name + '.s3.amazonaws.com/' + key)

NameError: name 'key' is not defined

In [ ]:
file_name = key.split('/')[-1].split('.')[0]
nc4_ds = netCDF4.Dataset(file_name, memory = resp.content)
store = xr.backends.NetCDF4DataStore(nc4_ds)
DS = xr.open_dataset(store)

In [ ]:
fig = plt.figure(figsize=(12, 12))
plt.imshow(DS.Rad, cmap='gray')
plt.axis('off')
plt.savefig(file_name + '.png', dpi=300, facecolor='w', edgecolor='w')